In [1]:
using CSV, DataFrames, Statistics, Clustering, Plots, Random, Gurobi, JuMP, DataStructures, StatsBase

In [2]:
data_prev = CSV.read("data/pred_Maria.csv",DataFrame)
data_opti = CSV.read("data/opti_Maria.csv",DataFrame)
data_logistic = CSV.read("data/opti_Maria.csv",DataFrame)
pred_logistic = CSV.read("data/pred_lr.csv",DataFrame)[:, "0"];

In [4]:
#number booked appointments
na = nrow(data_prev)
#number new appointments
nb = nrow(data_opti)
ntot = na + nb
#number doctors
nd = 15
#penalty added nurse
pen_nurse = 5
#penalty hole
pen
#previously_booked
s = data_prev.slot .+1 ;

In [13]:
na_1 = floor(na/15)
nb_1 = floor(nb/15)
data_prev_1 = data_prev[rand(1:na, Int(na_1)),:]
data_opti_1 = data_opti[rand(1:nb, Int(nb_1)),:];

In [14]:
function optim_add_nurse_hole(p)
model = Model(with_optimizer(Gurobi.Optimizer, TimeLimit=180))
set_optimizer_attribute(model, "OutputFlag", 0)

@variable(model, x[1:ntot, 1:50], Bin)
@variable(model, t[1:50])

@objective(model, Min, sum(t)*pen_nurse)

@constraint(model, [j=1:50], t[j] >= sum(x[i,j]*p[i,j] for i=1:ntot) - nd) 
@constraint(model, [j=1:50], t[j] >= 0)

for i=1:na
    for j=1:50
        if s[i]==j
            @constraint(model, x[i,j] == 1)
        else
            @constraint(model, x[i,j] == 0)
        end
    end
end

for i=na+1:ntot
    for j=1:50
        if j in a[i-na,:]
        else
            @constraint(model, x[i,j] == 0)
        end
    end
end

@constraint(model, [i=1:ntot], sum(x[i,j] for j=1:50) == 1)
        

optimize!(model)
return objective_value(model), JuMP.value.(x)
end

optim_add_nurse_hole (generic function with 1 method)